In [1]:
import os
with open('/Users/geoff/Downloads/TEST_ENCRYPT.png', 'rb') as img:
    img_bin = img.read()

In [2]:
num_img = len(img_bin)

In [3]:
incrementor = 2
check = False
presets = []

while incrementor < num_img + 1:
    if num_img % incrementor != 0:
        incrementor += 1
    else:
        presets.append(incrementor)
        incrementor += 1
print(presets)

[11, 31081, 341891]


In [4]:
num_img//11

31081

In [5]:
split_img = img_bin.split(maxsplit=11)
print(len(split_img))
split_img[3]

12


b'IHDR\x00\x00\x02\x08\x00\x00\x01\xa0\x08\x06\x00\x00\x00\xcf[\x9f}\x00\x00\x00\x19tEXtSoftware\x00Adobe'

### Private Key Generation, Saving key and loading key ###

In [6]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_pem_private_key


def gen_key():
    private_key = rsa.generate_private_key(
        public_exponent=65537, key_size=2048, backend=default_backend()
    )
    return private_key


def save_key(pk, filename):
    pem = pk.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    )
    with open(filename, 'wb') as pem_out:
        pem_out.write(pem)


def load_key(filename):
    with open(filename, 'rb') as pem_in:
        pemlines = pem_in.read()
    private_key = load_pem_private_key(pemlines, None, default_backend())
    return private_key


if __name__ == '__main__':
    pk = gen_key()
    filename = 'privkey.pem'
    save_key(pk, filename)
    pk2 = load_key(filename)
    pk3 = load_key(filename)

### Signing a private message

In [7]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
filename = 'privkey.pem'
private_key = load_key(filename)
message = b"A message I want to sign"
signature = private_key.sign(
     message,
     padding.PSS(
         mgf=padding.MGF1(hashes.SHA256()),
         salt_length=padding.PSS.MAX_LENGTH
     ),
     hashes.SHA256()
 )


In [8]:
signature

b'\x82\t\x98,\x13(\xdc\x00m\xf9\xb5\xaa\x9b\xfe\xe4~\xf7\x82\xac\x12\xd3\x95\xfeG\x1b\x18\xa8B\xccq\x1d\xf0\xf73\xd0\x1f\xdc\xde|\x8f\xc2\xb2\xcb\xb1-\xaa>d\x9aYI>jt\xbf\xe3\xa6\x1f\x14\xb5R\xb9(\x88\x05D\xfc@.XQ_,\x86\xc7\xe9\x1b\x8cHY\xf6y\xa7\x82\xe4\xfe\xaf\x8a=\xd1O\xc2\xa2\x1c\xe8\x11.\xb4\x03\xa6\xe0\xf4Kt\xaflI\xfe\x91\xe7\x13\x9d\xd7\x1d\x1d2Z~&\x05g\xf2\x13>;\xf7\xe1p]\x8e\x0e\xf6\x0co\xcfav_&\xe8\x82\x11S\x15\x1d5\x17\xe5\x8fD\x15\x1e\x99\xd2^u\xda`\xc5\xb6S\xfb*\xf9-\xbf|^N\xf6lt\xe1\x8f\xbb3n\xfe\r`\xe0 >\x10h\x1c\xd9\\L*\x8b\xf6\x8dI\xa3\xb2z\xab\xa3\xa1\x98\x0e\xcex\x17]!?\x1c\xf2-\xc9JS\xe41\x14\xf6u\x95a\x85\xa06\x05\xec\xb9\xf1\x99\xc9\xb3\xf8\xe7\xa1\x1b(O,\x8021\x06B\x906}\xfb\xe6A\tj\x9b\xb9\x9d\xd3\x1f'

### Verifying The signature ###

In [9]:
public_key = private_key.public_key()
print(public_key.verify(
     signature,
     message,
     padding.PSS(
         mgf=padding.MGF1(hashes.SHA256()),
         salt_length=padding.PSS.MAX_LENGTH
     ),
     hashes.SHA256()
 ))

None


### Encryption ###

In [10]:
 for message in split_img:
    ciphertext = public_key.encrypt(
         message,
         padding.OAEP(
             mgf=padding.MGF1(algorithm=hashes.SHA256()),
             algorithm=hashes.SHA256(),
             label=None
         )
     )
    print(ciphertext)
    print("==========================================================")

b'\x06]\xec\xac\x06sP{e`\xb9&OS\xf2!\x01\xb2x\x01d3\n\xdfL\x8f3\xb2\nP\x85\xa94\xdb\x88\x18c\xc0\x95\x06/\x0e\xf1\x9cu\xd9p>/\x88G<\x8f\x80\x15\xc3\\\xb3\x91\xd8\x03\x97\x91\x8box\xd8m\xba\xfa\x18\xa9\x18\xb0~\xadA:\xe3\'\xf5i\xc9\xfa\x17\x8a\x7fZ>mkX\x0e\x8aCw\x15\xf0\xab\xab\xc0\xb9fq\x1as0C\xfa\xc9\xfd\xe2\xd98\xcafS\xdb\x13M#\xf6\x0e\xf7q\x94\xf8Vu\xf0\xd3\xefD\x04W\xffl\xf4\xfc\x99\x9fWy\xb1\n\xeb\xfb@C~\xd3{\x9d7~\xadZ)\xd7vd!\xda4%\x02\xf6@\t+i\xb7\x14\xa4\xd5\xc9HyH7v\x93*\x07\xa8h\xcd\xe8#\x80\r\x10\xc9\xbd\xb5\x92%\x906\x8e\xd3\x1b\xff\xc9V\x90\xf7uZ\x04\xca\x14\x0cq\x19F\x17\x0f~\xaf\xc3\xb5;\xfcz\xc7\xcam\xe1c"\xb0\x02\xae\\\x19\xc1I\x91\xd6\xd4!\x9bQ\xbc\x18I\x93\\y\'\x96\xd4\xf0\x96F'
b'v/\xe9\x99\xe9EN\xdc\xc4\xa5\x80\x1db\x07[\xa3\x89\x9c\xe5\xf7[\x81\xa6\xc5O\x99!\x00\xc6y\xa5\x10.d\x1b\x88\xe1\xaam\xf4R\xf6\xf4\xff\xf6dQ\xae\'\x81<\x06\xbd\x83\xd6\xd8\x14\x80\x8b\xa54\xdf\x8c|\xc5d8\xc4\x81\xdf2\x16CC\xd1R!\x96\xd9\xd6^\xd6\x9c_\xc6\x1d\xb1\xed\n\xe8%7\x10!\xd9\x9fZD\

ValueError: Data too long for key size. Encrypt less data or use a larger key size.

### Saving to gleipnir file format ###

In [ ]:
encrypted_file = "block_01.glp"
with open(encrypted_file, 'wb') as file_out:
    file_out.write(ciphertext)

### Loading Gleipnir's file and decrypting it ###

In [ ]:
with open(encrypted_file, 'rb') as file_in:
    encrypted_message = file_in.read()

plaintext = private_key.decrypt(
     encrypted_message,
     padding.OAEP(
         mgf=padding.MGF1(algorithm=hashes.SHA256()),
         algorithm=hashes.SHA256(),
         label=None
     )
 )
plaintext == message
plaintext

### Putting Everything Together ###

In [ ]:
class GleipnirError(Exception):
    def __init__(self, message, errors):

        # Call the base class constructor with the parameters it needs
        super().__init__(message)

        # Now for your custom code...
        self.errors = errors

class Gleipnir(object):
    def __init__(self, private_key, args):
        self.private_key = private_key
        self.args = args
        
        self.file = None
        self.file_bin = None
        self.split = None
        self.output = None
        
    def get_file(self):
        if self.args.file is not None:
            self.file = self.args.file
            with open(self.file, 'rb') as img:
                self.file_bin = img.read()
            return self.file_bin
        return None
    
    def get_split(self, low=False, medium=False, high=False):
        if self.args.split is not None:
            self.split = self.args.split
            return self.split
        else:
            num_img = len(self.file_bin)
            if self.file_bin is not None:
                incrementor = 2
                check = False
                while not check:
                    if num_img % incrementor != 0:
                        print(incrementor)
                        incrementor += 1
                    else:
                        
                        check = True
                        self.split = incrementor
                return self.split
            else:
                raise GleipnirError("No file has been found...", {"file": "file not found"})
                
    def _split_file(self):
        if self.split is not None:
            split_img = self.file_bin.split(maxsplit=int(self.split))
            return split_img
        return None

    def get_public_key(self):
        return self.private_key.public_key()
        
    def encrypting_blocks(self):
        public_key = self.get_public_key()
        split_file = _self._split_file()
        if split_file is not None:
            if args.output is not None:
                output = args.output
                for increment, text_block in enumerate(split_file):
                    ciphertext = public_key.encrypt(
                         text_block,
                         padding.OAEP(
                             mgf=padding.MGF1(algorithm=hashes.SHA256()),
                             algorithm=hashes.SHA256(),
                             label=None
                         )
                     )
                    if output.endswith(".glp"):
                        encrypted_file = os.path.join(os.path.dirname(output), "block_{:>04d}.glp".format(increment))
                        with open(encrypted_file, 'wb') as file_out:
                            file_out.write(ciphertext)

In [ ]:
filename = 'privkey.pem'
private_key = load_key(filename)

import argparse


def get_gleipnir_args():
    parser = argparse.ArgumentParser()
    main_group = parser.add_argument_group("Main options")
    main_group.add_argument("-f", "--file", action="store",
                            help="Get the path of the file.")
    main_group.add_argument("-s", "--split", action="store",
                            help="force the number of blocks that split "
                            "the file.")
    main_group.add_argument("-o", "--output", action="store",
                            help="specify the output path of the blocks.")
    return parser.parse_args()

In [ ]:
import Crypto
help(Crypto)